In [ ]:
import math
import numpy as np
import gerrychain
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import geopandas as gpd
import pandas as pd
import gurobipy as gp
import csv
import time
import json
import sys
import os

from datetime import date
from gurobipy import GRB 
from gerrychain import Graph
# Reading in our shapefile
fp = "/Users/patrickcummins/Desktop/Thesis/Data/County Data Files/North_Carolina_State_and_County_Boundary_Polygons/North_Carolina_State_and_County_Boundary_Polygons.shp"
data2 = gpd.read_file(fp)
# Reading in demographics information

In [ ]:
fp2 = "/Users/patrickcummins/Desktop/Thesis/split_counties.shp"
data = gpd.read_file(fp2)

In [ ]:
def update_county_names(dataa, county_name):
    # Filter the GeoDataFrame for the specific county
    county_rows = data[data['County'] == county_name]
    # Generate new names based on the number of parts
    new_names = [f"{county_name}{i + 1}" for i in range(len(county_rows))]
    # Assign new names to the 'County' column
    data.loc[county_rows.index, 'County'] = new_names

# Update names for each divided county
divided_counties = ['Wake', 'Mecklenburg']  # List of counties you divided
for county in divided_counties:
    update_county_names(data, county)

# Check the updated GeoDataFrame
print(data[['County']])

In [ ]:
import matplotlib.pyplot as plt
data.plot(column='County')
plt.show()

In [ ]:
import pandas as pd
import geopandas as gpd
from gerrychain import Graph

# Load demographic data
demographics = pd.read_csv('/Users/patrickcummins/Desktop/Thesis/Data/Demographics.csv')
demographics_to_merge = demographics[['County Name', 'Population']]
demographics_to_merge = demographics_to_merge.rename(columns={'County Name': 'County', 'Population': 'Population'})
demographics_to_merge['Population'] = demographics_to_merge['Population'].str.replace(',', '').astype(int)

# Merge demographic data
data = data.merge(demographics_to_merge, on='County', how = 'left')
data['County'] = data['County'].str.upper()
data

In [ ]:
wake_pop_new = round(float(1189705 / 4))
meck_pop_new = round(float(1159791 / 4))

In [ ]:
split_counties = ['WAKE1', 'WAKE2', 'WAKE3', 'WAKE4', 'MECKLENBURG1', 'MECKLENBURG2', 'MECKLENBURG3', 'MECKLENBURG4']
# Adjust population for split counties
split_counties_wake = ['WAKE1', 'WAKE2', 'WAKE3', 'WAKE4']
split_counties_meck = 'MECKLENBURG1', 'MECKLENBURG2', 'MECKLENBURG3', 'MECKLENBURG4'
data.loc[data['County'].isin(split_counties_wake), 'Population'] = wake_pop_new
data.loc[data['County'].isin(split_counties_meck), 'Population'] = meck_pop_new
data

In [ ]:
# Load voting data
voting = pd.read_csv("/Users/patrickcummins/Desktop/Thesis/Data/Voting.csv")
voting = voting.iloc[1:-1]
voting = voting.rename(columns={'Unnamed: 4': 'Other Affiliation'})

# Merge voting data
data = data.merge(voting, on='County', how = 'left')

# Adjust voting counts for split counties
voting_columns = ['Democratic', 'Republican', 'Total']
for column in voting_columns:
    data.loc[data['County'].isin(split_counties), column] /= 4
# Convert the GeoDataFrame to a GerryChain Graph
graph = Graph.from_geodataframe(data)

In [ ]:
voting.loc[voting['County'] == 'MECKLENBURG']

In [ ]:
wake_dems = round(float(286077 / 4))
wake_reps = round(float(176234 / 4))
wake_total = round(float(821613 / 4))
wake_other = round(float(359302 / 4))

meck_dems = round(float(322505 / 4))
meck_reps = round(float(155784 / 4))
meck_total = round(float(788310 / 4))
meck_other = round(float(310021 / 4))

data.loc[data['County'].isin(split_counties_wake), 'Democratic'] = wake_dems
data.loc[data['County'].isin(split_counties_wake), 'Republican'] = wake_reps
data.loc[data['County'].isin(split_counties_wake), 'Total'] = wake_total
data.loc[data['County'].isin(split_counties_wake), 'Other Affiliation'] = wake_other

data.loc[data['County'].isin(split_counties_meck), 'Democratic'] = meck_dems
data.loc[data['County'].isin(split_counties_meck), 'Republican'] = meck_reps
data.loc[data['County'].isin(split_counties_meck), 'Total'] = meck_total
data.loc[data['County'].isin(split_counties_meck), 'Other Affiliation'] = meck_other

data

In [ ]:
# Convert the GeoDataFrame to a GerryChain Graph
graph = Graph.from_geodataframe(data)
# Add population data to the county nodes
for node in graph.nodes():
    # Match the node with the geodataframe index
    graph.nodes[node]['population'] = data.loc[node, 'Population']
    graph.nodes[node]['county'] = data.loc[node, 'County']
    graph.nodes[node]['geoid'] = data.loc[node, 'OBJECTID']
    graph.nodes[node]['num_democrats'] = data.loc[node, 'Democratic']
    graph.nodes[node]['num_republicans'] = data.loc[node, 'Republican']
    graph.nodes[node]['voting_population'] = data.loc[node, 'Total']
    # create model 
deviation = 0.01

import math
k = 4
total_population = sum(graph.nodes[node]['Population'] for node in graph.nodes)
population = [graph.nodes[i]['Population'] for i in graph.nodes()]    
L = math.ceil((1-deviation/2)*total_population/k)
U = math.floor((1+deviation/2)*total_population/k)
print("Using L =",L,"and U =",U,"and k =",k)
sigma = 0.1
DG = nx.DiGraph(graph)

In [ ]:
m = gp.Model()
x = m.addVars(graph.nodes, k, vtype = GRB.BINARY)
y = m.addVars(graph.edges, vtype = GRB.BINARY)
z = m.addVars(k, vtype = GRB.CONTINUOUS)
alpha = 0.5 # EDIT THESE TO CHANGE IMPORTANCE OF OBJECTIVE FUNCTION
beta = 0.5

m.setObjective(alpha * gp.quicksum(y[u,v] for u,v in graph.edges) + beta * gp.quicksum(z[j] for j in range(k)), GRB.MINIMIZE)

m.addConstrs(gp.quicksum(x[i,j] for j in range(k)) == 1 for i in graph.nodes)

m.addConstrs(gp.quicksum(x[i,j] * graph.nodes[i]['population'] for i in graph.nodes) >= L for j in range(k))
m.addConstrs(gp.quicksum(x[i,j] * graph.nodes[i]['population'] for i in graph.nodes) <= U for j in range(k))

m.addConstrs(z[j] >= gp.quicksum(x[i, j] * graph.nodes[i]['num_republicans'] for i in graph.nodes) - 
             gp.quicksum(x[i, j] * graph.nodes[i]['num_democrats'] for i in graph.nodes) for j in range(k))
m.addConstrs(z[j] >= gp.quicksum(x[i, j] * graph.nodes[i]['num_democrats'] for i in graph.nodes) - 
             gp.quicksum(x[i, j] * graph.nodes[i]['num_republicans'] for i in graph.nodes) for j in range(k))


# add constraints saying that edge {i,j} is cut if i is assigned to district v but j is not.
m.addConstrs( x[i,v] - x[j,v] <= y[i,j] for i,j in graph.edges for v in range(k))

r = m.addVars(graph.nodes, k, vtype=GRB.BINARY)

# Add flow variables: f[u,v] = amount of flow sent across arc uv 
#  Flows are sent across arcs of the directed version of G which we call DG
import networkx as nx
DG = nx.DiGraph(graph) # directed version of G
f = m.addVars(DG.edges, vtype=GRB.CONTINUOUS)
# The big-M proposed by Hojny et al.
M = graph.number_of_nodes() - k + 1

# Each district j should have one root
m.addConstrs( gp.quicksum( r[i,j] for i in DG.nodes) == 1 for j in range(k) )

# If node i is not assigned to district j, then it cannot be its root
m.addConstrs( r[i,j] <= x[i,j] for i in DG.nodes for j in range(k) )  

# if not a root, consume some flow.
# if a root, only send out (so much) flow.
m.addConstrs( gp.quicksum( f[u,v] - f[v,u] for u in DG.neighbors(v) ) >= 1 - M * gp.quicksum( r[v,j] for j in range(k)) for v in graph.nodes)

# do not send flow across cut edges
m.addConstrs( f[i,j] + f[j,i] <= M * (1 - y[i,j]) for (i,j) in graph.edges )
m.update()

In [ ]:
m.optimize()

In [ ]:
# retrieve the districts and their populations
districts = [ [i for i in graph.nodes if x[i,j].x > 0.5] for j in range(k)]
district_counties = [ [ graph.nodes[i]['county'] for i in districts[j] ] for j in range(k)]
district_populations = [ sum(graph.nodes[i]["population"] for i in districts[j]) for j in range(k) ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",district_counties[j])

In [ ]:
# Which district is each county assigned to?
assignment = [ -1 for u in graph.nodes ]
    
# for each district j
for j in range(len(districts)):
    
    # for each node i in this district
    for i in districts[j]:
        
        # What is its GEOID?
        geoID = graph.nodes[i]["OBJECTID"]
        
        # Need to find this GEOID in the dataframe
        for u in graph.nodes:
            if geoID == data['OBJECTID'][u]: # Found it
                assignment[u] = j # Node u from the dataframe should be assigned to district j

# Now add the assignments to a column of the dataframe and map it
data['assignment'] = assignment
my_fig = data.plot(column='assignment').get_figure()

In [ ]:
assignments = {i: j for i in graph.nodes for j in range(k) if x[i, j].X > 0.5}

# Initialize counters for each district
total_democrats = {j: 0 for j in range(k)}
total_republicans = {j: 0 for j in range(k)}
total_voters = {j: 0 for j in range(k)}

# Sum up the numbers based on assignments
for i in graph.nodes:
    district_assigned = assignments[i]
    total_democrats[district_assigned] += graph.nodes[i]['num_democrats']
    total_republicans[district_assigned] += graph.nodes[i]['num_republicans']
    total_voters[district_assigned] += graph.nodes[i]['num_democrats'] + graph.nodes[i]['num_republicans']

# Calculate proportions for each district
proportions = {}
for j in range(k):
    if total_voters[j] > 0:  # Prevent division by zero
        sum_dem = total_democrats[j]
        sum_rep = total_republicans[j]
        sum_total = total_voters[j]
        prop_democrats = total_democrats[j] / total_voters[j]
        prop_republicans = total_republicans[j] / total_voters[j]
        proportions[j] = {'Democrats': prop_democrats, 'Republicans': prop_republicans}
        difference = prop_democrats / prop_republicans
    else:
        proportions[j] = {'Democrats': 0, 'Republicans': 0}

# Output the proportions
for j in proportions:
    print(f"District {j}: Democrats: {proportions[j]['Democrats']:.2f}, Republicans: {proportions[j]['Republicans']:.2f}")
    